In [1]:
import pyspark
import os
os.getcwd()
import findspark
findspark.init('/Users/priyankapatil/anaconda3/lib/python3.7/site-packages/pyspark')
print ("Hey Priyanka")
from pyspark.sql import Row
from pyspark.sql import functions
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark.sql.functions as func
from pyspark.sql import SparkSession

spark = SparkSession.builder.config("spark.sql.warehouse.dir", "file:///C:/temp").appName("SparkSQL").getOrCreate()

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
sqlContext = SQLContext(sc)
sc

Hey Priyanka


<SparkContext master=local[*] appName=SparkSQL>

In [18]:
import pandas as pd
del data
alldfs = [var for var in dir() if isinstance(eval(var), pd.core.frame.DataFrame)]
print(alldfs)

['data_cancelled', 'pandas_datacancelled']


In [4]:
data = pd.read_csv(r'/Users/priyankapatil/Desktop/Data/dataset_flight.csv',sep=',')


/Users/priyankapatil/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (0,1,2,3,4,7,10,11,12,13,14,15,16,17,18,20,21,22,23,24,25,26,27,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
data.drop(["DEP_TIME","ARR_TIME","TAIL_NUM","TAXI_OUT","TAXI_IN","YEAR", "QUARTER","DAY_OF_MONTH","AIR_TIME","OP_UNIQUE_CARRIER"], axis=1, inplace=True)

In [6]:
#Filtering only flights that were cancelled
data_cancelled=data.query("CANCELLED == 1")

In [7]:
data_cancelled.shape

(236794, 21)

In [9]:
#Converting columns to float
int_cols = ['MONTH','DAY_OF_WEEK','OP_CARRIER_FL_NUM','CRS_DEP_TIME','DEP_DELAY','CRS_ARR_TIME']
data_cancelled[int_cols] = data_cancelled[int_cols].apply(pd.to_numeric, downcast='float', errors='coerce')
int_cols = ['CANCELLED','DIVERTED','CRS_ELAPSED_TIME','ARR_DELAY']
data_cancelled[int_cols] = data_cancelled[int_cols].apply(pd.to_numeric, downcast='float', errors='coerce')

In [10]:
int_cols = ['ACTUAL_ELAPSED_TIME','DISTANCE','CARRIER_DELAY','WEATHER_DELAY','NAS_DELAY','SECURITY_DELAY','LATE_AIRCRAFT_DELAY']
data_cancelled[int_cols] = data_cancelled[int_cols].apply(pd.to_numeric, downcast='float', errors='coerce')


In [11]:
data_cancelled.dtypes

MONTH                  float32
DAY_OF_WEEK            float32
OP_CARRIER_FL_NUM      float32
ORIGIN                  object
DEST                    object
CRS_DEP_TIME           float32
DEP_DELAY              float32
CRS_ARR_TIME           float32
ARR_DELAY              float32
CANCELLED              float32
CANCELLATION_CODE       object
DIVERTED               float32
CRS_ELAPSED_TIME       float32
ACTUAL_ELAPSED_TIME    float32
DISTANCE               float32
CARRIER_DELAY          float32
WEATHER_DELAY          float32
NAS_DELAY              float32
SECURITY_DELAY         float32
LATE_AIRCRAFT_DELAY    float32
Unnamed: 30            float64
dtype: object

In [13]:
import numpy as np
data_cancelled['DEP_DEL15'] = np.where((data_cancelled['DEP_DELAY'] > 15),1,0)


/Users/priyankapatil/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
int_cols = ['DEP_DEL15']
data_cancelled[int_cols] = data_cancelled[int_cols].apply(pd.to_numeric, downcast='float', errors='coerce')

In [15]:
data_cancelled=data_cancelled.loc[:, ~data_cancelled.columns.str.contains('^Unnamed')]

In [16]:
pandas_datacancelled=data_cancelled

In [19]:
#Create Schema to convert to Spark dataframe
schema = StructType([StructField("Month", StringType(), True)\
                       ,StructField("DAY_OF_WEEK", FloatType(), True)\
                       ,StructField("FlightNo", FloatType(), True)\
                       ,StructField("Origin", StringType(), True)\
                       ,StructField("Dest", StringType(), True)\
                       ,StructField("SchDep", FloatType(), True)\
                       ,StructField("DepDelay", FloatType(), True)\
                       ,StructField("SchArr", FloatType(), True)\
                       ,StructField("ArrDelay", FloatType(), True)\
                       ,StructField("Cancelled", FloatType(), True)\
                       ,StructField("Cancelcode", StringType(), True)\
                       ,StructField("Diverted", FloatType(), True)\
                       ,StructField("SchElapsedTime", FloatType(), True)\
                       ,StructField("ElapsedTime", FloatType(), True)\
                       ,StructField("Distance", FloatType(), True)\
                       ,StructField("CarrierDelay", FloatType(), True)\
                       ,StructField("WeatherDelay", FloatType(), True)\
                       ,StructField("NASDelay", FloatType(), True)\
                       ,StructField("SecurityDelay", FloatType(), True)\
                       ,StructField("LateAircraftDelay", FloatType(), True)\
                       ,StructField("DepDelay15", FloatType(), True)])

In [20]:
dataspark = spark.createDataFrame(pandas_datacancelled, schema=schema)
dataspark.show(5)


+-----+-----------+--------+------+----+------+--------+------+--------+---------+----------+--------+--------------+-----------+--------+------------+------------+--------+-------------+-----------------+----------+
|Month|DAY_OF_WEEK|FlightNo|Origin|Dest|SchDep|DepDelay|SchArr|ArrDelay|Cancelled|Cancelcode|Diverted|SchElapsedTime|ElapsedTime|Distance|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|DepDelay15|
+-----+-----------+--------+------+----+------+--------+------+--------+---------+----------+--------+--------------+-----------+--------+------------+------------+--------+-------------+-----------------+----------+
|  4.0|        2.0|   856.0|   LAX| IAD| 947.0|     NaN|1748.0|     NaN|      1.0|         A|     0.0|         301.0|        NaN|  2288.0|         NaN|         NaN|     NaN|          NaN|              NaN|       0.0|
|  4.0|        2.0|   421.0|   ATL| EWR|1036.0|     NaN|1255.0|     NaN|      1.0|         A|     0.0|         139.0|        NaN|   

In [21]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col

In [23]:
from pyspark.ml.feature import StringIndexer

stringIndexer = StringIndexer(inputCol="SchDep", outputCol="index")
model = stringIndexer.fit(dataspark)
df_data = model.transform(dataspark)

In [24]:
df_data.rdd

MapPartitionsRDD[22] at javaToPython at NativeMethodAccessorImpl.java:0

In [25]:
#parse columns for classification(data) & Regression(datar)

data = df_data.rdd.map(lambda x: ((x[10]), Vectors.dense(float(x[2]),float(x[1]),float(x[5]),float(x[7]),float(x[14]))))
datar = df_data.rdd.map(lambda x: ((x[9]), Vectors.dense(float(x[2]),float(x[1]),float(x[5]),float(x[7]),float(x[14]))))
datalr = df_data.rdd.map(lambda x: ((x[9]), Vectors.dense(float(x[5]))))

In [26]:
from pyspark.ml.linalg import Vectors

datap = sqlContext.createDataFrame(data, ['label','features'],samplingRatio=0.2)
datapr = sqlContext.createDataFrame(datar, ['label','features'],samplingRatio=0.2)

In [27]:
datlr = sqlContext.createDataFrame(datalr, ['label','features'],samplingRatio=0.2)


In [28]:
#Filter out null values

dfapp1=datap.filter("label is not null")
dfapp2=datapr.filter("label is not null")

In [29]:
dfapp3=datlr.filter("label is not null")


In [30]:

labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel", handleInvalid="skip").fit(dfapp1)

In [31]:

featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(dfapp1)

In [32]:
# Split the data into training and test sets 70 and 30
(trainingData, testData) = dfapp1.randomSplit([0.7, 0.3])

In [33]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString

# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)

In [34]:
# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

In [35]:
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

In [36]:
# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

In [48]:
# Make predictions.
predictions = model.transform(testData)

In [38]:
# Select example rows to display.
predictions.select("predictedLabel", "label", "features").show(5)

+--------------+-----+--------------------+
|predictedLabel|label|            features|
+--------------+-----+--------------------+
|             B|    A|[1.0,1.0,800.0,11...|
|             B|    A|[1.0,3.0,700.0,80...|
|             B|    A|[2.0,4.0,630.0,73...|
|             B|    A|[2.0,4.0,900.0,17...|
|             B|    A|[3.0,3.0,730.0,83...|
+--------------+-----+--------------------+
only showing top 5 rows



In [39]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.429697


In [41]:
rfModel = model.stages[2]
print(rfModel)

RandomForestClassificationModel (uid=RandomForestClassifier_8177945ba633) with 10 trees
